In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [24]:
df_real = pd.read_csv("winequality-red.csv")
df = df_real
df.head();

In [25]:
df.isnull().sum();

In [26]:
df.dtypes;

In [27]:
df = df.drop_duplicates()

In [28]:
# Multicolinearity check krne keliye
'''
corr = df.corr()
plt.figure(figsize=(12, 6))  
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.plot()
'''

"\ncorr = df.corr()\nplt.figure(figsize=(12, 6))  \nsns.heatmap(corr, annot=True, cmap='coolwarm')\nplt.plot()\n"

# Ye 5 cheezein dhyn rkhna h 
Linearity – X aur y ka relationship linear ho.

Independence – errors independent ho.

Homoscedasticity – errors ki spread constant ho.

No multicollinearity – features ek dusre ke duplicate na ho.

Normality of errors – errors bell-curve jaisa ho.

In [29]:
#outliners pta krne keliye
'''
plt.figure(figsize=(12,6))
sns.boxplot(data = df)
'''

'\nplt.figure(figsize=(12,6))\nsns.boxplot(data = df)\n'

In [30]:
# yha se kitna data h pta kiya
df.describe();

In [31]:
df.var();
#density danger me h because variance bhot kam h data me

In [32]:
# to check linearity with each feature 
'''
target = 'quality'
for col in df.columns:
    if col != target:
        sns.scatterplot(x=df[col], y=df[target])
        plt.show()
'''

"\ntarget = 'quality'\nfor col in df.columns:\n    if col != target:\n        sns.scatterplot(x=df[col], y=df[target])\n        plt.show()\n"

In [33]:
df_clean = df.copy()
Q1 = df_clean.quantile(0.25)
Q3 = df_clean.quantile(0.75)
IQR = Q3 - Q1

# filter rows within [Q1 - 1.5*IQR, Q3 + 1.5*IQR] for all columns
df_clean = df_clean[~((df_clean < (Q1 - 1.5 * IQR)) | (df_clean > (Q3 + 1.5 * IQR))).any(axis=1)]
df = df_clean
df.shape


(1005, 12)

In [34]:
df = df.reset_index(drop = True)
df;

In [35]:
m = len(df)
label = df['quality']
feature = df.drop(columns='quality')

In [36]:
#test aur train data split krke usko array me change kr diya 
train_feature = feature[ : int(0.8*m)].values 
train_label = label[ : int(0.8*m)].values
test_feature = feature[int(0.8*m) : ].values
test_label = label[int(0.8*m) : ].values

In [37]:
scaler = StandardScaler()
train_feature = scaler.fit_transform(train_feature)
test_feature = scaler.transform(test_feature)
train_feature.shape

(804, 11)

In [38]:
# weight aur bais initially assign kr diya h sab features keliye
w= np.zeros(train_feature.shape[1])
b = 0
x = 0.005 # learning rate

In [39]:
# yha pe each feature apne weight ke saath align honge aur bias add krke return dega  
def prediction(train_feature,w,b):
    return np.dot(train_feature,w) + b

In [40]:
# cost function :: exactly error kya h predicted value aur current value ke beech wo btata h 
def cost_function(y_pre,train_label):
    m = len(train_label)
    return ((1 / (2 * m)) * np.sum((train_label - y_pre)**2))

In [41]:
# cost me kitna contribute kr rha h hr ek value of each featur usko dekhna h uske according weight update krenge  
def weight_contri(train_feature,train_label,w,x,b):
    m = len(train_label)
    y_pred = prediction(train_feature, w, b)
    error = y_pred - train_label

    dw = (1/m) * np.dot(train_feature.T, error)
    db = (1/m) * np.sum(error)

    w = w - x * dw
    b = b - x * db
    return w, b

In [42]:
epoch = 100000
cost_history = []
standard_error = 0.01

for i in range(epoch):
    y_pred = prediction(train_feature, w, b)
    cost = cost_function(y_pred, train_label)
    cost_history.append(cost)
    w, b = weight_contri(train_feature, train_label, w, x, b)

print(f"final weight : {w}")
print(f"final bais : {b}")

final weight : [ 0.0389146  -0.13143774 -0.04336926 -0.00614159 -0.01417637  0.0278977
 -0.07970853 -0.05072279 -0.0435044   0.19904287  0.25432674]
final bais : 5.630597014925282


In [43]:
cost_history;

In [44]:
y_test_pred = prediction(test_feature, w, b)
test_cost = cost_function(y_test_pred, test_label)
print(f"\nTest MSE: {test_cost:.4f}")

# Optional: R² Score
SS_res = np.sum((test_label - y_test_pred)**2)
SS_tot = np.sum((test_label - np.mean(test_label))**2)
r2_score = 1 - (SS_res / SS_tot)
print(f"R² Score on Test Data: {r2_score:.4f}")


Test MSE: 0.1754
R² Score on Test Data: 0.2767
